## Song lyric puller

I'm going to analyze the songs in my music library.

I've been using Google Play Music (GPM) for about 10 years now. I have all the albums I own uploaded to my digital library, I've made playlists for friends and family and myself, and collected all of the playlists of the radio show I used to host in grad school. Recent actions by Google have foreshadowed the shuddering of this service. They have a near identical service, Youtube Music, running in parallel to GPM. They appear to have decided Youtube Music will be there music streaming service and have been phasing GPM out. This puts my music and my playlists in jeoparady. So, I found a way to extract all that info to CSV files.

Now, I'd like to do some Natural Language Processing on this lists to glean some interesting info out of them. First however, I think if we are dealing with songs and NLP, we should collect the lyrics to those songs too.

This Webscraping routine pulls the lyrics of each of the songs in my library from Genius using selenium, parses it using Beautiful Soup, and adds it my master GPM list of songs using pandas.

In [1]:
#bring in all the libraries I'll need

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

import time
import string
import requests
from bs4 import BeautifulSoup

import unidecode
import pandas as pd

In [2]:
# set driver options and import song list
option = webdriver.ChromeOptions()
option.add_argument(' - incognito')

gpm=pd.read_csv('GPM_song_list.csv')

gpm.head(5)

,artist,album,title,duration,playcount,rating,rating_interpretation
0,Los Campesinos!,"Hold On Now, Youngster",... And We Exhale And Roll Our Eyes In Unison,2:49,1.0,0,not-rated
1,The Steinways,Missed The Boat,... But You're Really Nice,0:17,2.0,undefined,never-rated
2,Titus Andronicus,The Most Lamentable Tragedy,[ intermission ],1:17,1.0,undefined,never-rated
3,A Giant Dog,Pile,& Rock & Roll,2:56,36.0,5,thumbs-up
4,Titus Andronicus,The Most Lamentable Tragedy,[ seven seconds ],0:07,NaN,undefined,never-rated


In [50]:
#set up driver object and collect lyrics

driver = webdriver.Chrome('C:/chromedriver_win32/chromedriver.exe')
driver.implicitly_wait(5)

#pull band and song info needed to direct to correct webpage on genius and format it correctly, remove punctuation, 
#replace spaces with hyphons.

i=1200

band=gpm.loc[i][0]
band = unidecode.unidecode(band)
band=band.replace('&','and')
band=band.replace(' ','-')
band=band.replace('!','')
song=gpm.loc[i][2]
song = unidecode.unidecode(song)
song=song.replace('&','and')
song=song.replace('...','')
song=song.replace(' ','-')

try:
    driver.get(f'https://genius.com/{band}-{song}-lyrics')
    
    #pull lyrics from page

    page= driver.find_element_by_xpath(f'/html/body/routable-page/ng-outlet/song-page/div/div/div/div/div')

    soup=BeautifulSoup(page.text,'html.parser')

    #clean lyrics by changing soup object to list of strings, removing \n  and \u2005 characters, and non lyric strings
    #at the beginning and end of each page.

    s=str(soup)
    s=s.replace('\u2005',' ')
    s=s.split('\n')
    s.pop()
    s.pop()
    s.pop(0)

    # remove blank strings and strings containing song structure info.

    s=[x for x in s if '[' not in x]
    for line in s:
        if line == '':
            s.remove('')

    # concatenate the list of strings into a single string. 
    lyrics=''
    for line in s:        
        lyrics+=str(line)
        lyrics+=' '
    print(lyrics)

except:
    print('web page did not resolve')

finally:
    driver.quit()

web page did not resolve


The routine works for a single song, now write a loop that will iterate through the entire library. This is a slow process, and once in a while (with how tempormental Webdrivers tend to be) the program crashes. It continues to iterate through the loop, but does not continue to access webpages correctly.

In [4]:
lst=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
for e in lst:
    print(e*913)
    
print(len(gpm.index))

913
1826
2739
3652
4565
5478
6391
7304
8217
9130
10043
10956
11869
12782
13695
14608
15521
16434
17347
18260
18266


In [6]:
18266-14608

3658

In [51]:
RAW=[]
URL=[]
driver = webdriver.Chrome('C:/chromedriver_win32/chromedriver.exe')

#pull band and song info needed to direct to correct webpage on genius and format it correctly
error = 0
for i in range(17808,18266):
    
    band=gpm.loc[i][0]
    
    band=band.replace('&','and')
    band=band.replace(' ','-')
    band=band.replace('!','')
    band=band.replace('?','')
    band=band.replace(',','')
    band=band.replace("'",'')
    band=band.replace('"','')
    band=band.replace(':','')
    band=band.replace(';','')
    band = unidecode.unidecode(band)
    
    song=gpm.loc[i][2]
    
    song=song.replace('&','and')
    song=song.replace('...','')
    song=song.replace('(','')
    song=song.replace(')','')
    song=song.replace(' ','-')
    song=song.replace(',','')
    song=song.replace("'",'')
    song=song.replace('"','')
    song=song.replace(':','')
    song=song.replace(';','')
    song = unidecode.unidecode(song)

    url=f'https://genius.com/{band}-{song}-lyrics'
    
    try:
        #access page
        
        time.sleep(2)
        driver.get(url)
        
        #pull lyrics from page
        
        time.sleep(2)
        page=driver.find_element_by_class_name('lyrics')
        soup=BeautifulSoup(page.text,'html.parser')

        #clean lyrics by changing soup object to list of strings, removing \n  and \u2005 characters, and non lyric strings
        #at the beginning and end of each page.

        s=str(soup)
        s=s.replace('\u2005',' ')
        s=s.split('\n')
        s.pop()
        s.pop()
        s.pop(0)

        # remove blank strings and concatenate the list of strings into a single string.

        s=[x for x in s if '[' not in x]
        for line in s:
            if line == '':
                s.remove('')
            elif line== 'SEE MORE':
                s.remove('SEE MORE')

        # concatenate the list of strings into a single string. 
        lyrics=''
        for line in s:        
            lyrics+=str(line)
            lyrics+=' '
        #print(lyrics)
        RAW.append(lyrics)
        URL.append(url)
        print(i)
    except:
        error+=1
        print(f'Failed. url used was {url} error number {error}')
        lyrics='F'
        RAW.append(lyrics)
        URL.append(url)
        pass
        

print('done')
driver.quit()

Failed. url used was https://genius.com/Pish-With-You-lyrics error number 1
17809
17810
17811
17812
17813
17814
17815
17816
Failed. url used was https://genius.com/Fat-White-Family-Without-Consent-lyrics error number 2
Failed. url used was https://genius.com/Bonnie-Prince-Billy-Without-Word-You-Have-Nothing-lyrics error number 3
17819
17820
Failed. url used was https://genius.com/Catholic-Action-Witness-lyrics error number 4
17822
17823
17824
17825
Failed. url used was https://genius.com/M83-The-Wizard-lyrics error number 5
Failed. url used was https://genius.com/The-Flaming-Lips-The-Wizard-Turns-On-lyrics error number 6
Failed. url used was https://genius.com/Pearl-Jam-W.M.A.-lyrics error number 7
Failed. url used was https://genius.com/The-Aquadolls-Woah-Man-lyrics error number 8
17830
17831
17832
17833
17834
17835
Failed. url used was https://genius.com/Mr.-Gnome-Wolf-Girls-lyrics error number 9
17837
17838
17839
17840
17841
17842
17843
17844
Failed. url used was https://genius.com/

Failed. url used was https://genius.com/Shin-Guard-You-Will-Be-Held-Accountable-for-Your-Actions-lyrics error number 74
18093
18094
18095
18096
18097
18098
18099
18100
18101
18102
18103
18104
18105
18106
18107
Failed. url used was https://genius.com/Thunderpussy-Young-and-Pure-lyrics error number 75
18109
Failed. url used was https://genius.com/A.C.-Newman-Young-Atlantis-lyrics error number 76
Failed. url used was https://genius.com/Kishi-Bashi-Young-Bride-lyrics error number 77
18112
18113
18114
Failed. url used was https://genius.com/Makaya-McCraven-Young-Genius-lyrics error number 78
18116
18117
18118
18119
Failed. url used was https://genius.com/St.-Vincent-Young-Lover-lyrics error number 79
18121
Failed. url used was https://genius.com/The-Shins-Young-Pilgrim-[Acoustic-Version]-lyrics error number 80
18123
18124
18125
18126
18127
Failed. url used was https://genius.com/Run-For-Cover-Records-Young-Statues---California-Girls-lyrics error number 81
18129
18130
18131
18132
18133
18134

ran a test of 100 songs. It was necessary to tell the program to wait a few seconds for the page to load. I always find tell python on the command line to hold tends to work better with selenium. When trying to use seleniums more advanced wait actions it seems to fall over itself and crash. 

after 100 songs, I had 38 lyrics webpages that didn't resolve. I'd like them to all run but they fail for various reasons. I had the output of the routine print the url that was attempted to help troubleshoot why some of the urls didn't work. The colprite for many was commas and apostrophes in the song title that were not in the url. run the 100 song test again, removing them before being entered.

correcting the punctuation cut down the errors from 38 to 28. The rest are more unique issues that would take to long to address each issue for each song. The other situation is that genius doesn't carry the songs lyrics at all. 72% of 18000+ songs will still be plenty of text to do analysis on. 

now exporting the lyric information.

I have to believe the interplay between google chrome, selenium and by ram was causing issues. Dependent upon certain ads appearing, or me checking my email, etc. the routine would crash. Even if I was not doing other tasks, at points my routine would crash. The quickest way to fix this was then to grab the lyrics in shorter bunches. I did 200 or 400 song bunches.

If I were to rewrite this program I would reset the actions so that the driver is called to and quit out of in very small bunches, 5 10 or 20 songs. The that set would be set aside in a collection variable, and the process would repeat. That way if a crash did happen, it would only be 5 to 20 songs and the majority of the collected data would be secure.

In [52]:
combine=combine+RAW

In [53]:
len(combine)

4571

In [54]:
part4=pd.DataFrame(combine)
part4.to_csv('working4.csv')

In [72]:
df1=pd.read_csv('working.csv',index_col='Unnamed: 0')
df2=pd.read_csv('working2.csv',index_col='Unnamed: 0')
df3=pd.read_csv('working3.csv',index_col='Unnamed: 0')
df4=pd.read_csv('working4.csv',index_col='Unnamed: 0')

In [77]:
df=df1.append(df2)

In [81]:
df=df.append(df4)

Here I take the various working files that I've collected lyrics in as the finiky program was wrestled with and combine them into a single csv

In [100]:
df.to_csv('lyrics.csv',index=False)

check the succesful capture rate

In [98]:

(len(df[df['0'].values == 'F']))/df['0'].count()

0.30989469065379555

a little over 30% failure on pulling lyrics. As I watched the failure urls, some were obvious. I could have used a string.punctuation to remove all punctuation from the url, but there were cases where certain punctuation remaing in the url. Instead I hand picked the most common errors I was seeing and the corrections that allowed them to resolve. There were unique situations that would have needed special treatment.

another category of misses were songs that didn't have lyrics pages because they were live versions, bootlegs, or podcasts! So those misses are acceptable.

A third category were small bands and artists that were not on genius. The only way to find the lyrics would be to hand search many would be on the bandcamp, but lyrics are not always posted, even there. 

For now, I want to build a SQL database, one that synthesizes my playlists song lists and lyrics. Remove duplicates and export an CSV that will have just the info I want. 

# Preliminary data cleaning

as I was running through the song list I found dubplicates and podcasts. I need to remove those entires and the lyrics for them.

In [8]:
gpm=pd.read_csv('GPM_song_list.csv')

lyrics=pd.read_csv('lyrics.csv')

In [9]:
gpm.head()

,artist,album,title,duration,playcount,rating,rating_interpretation
0,Los Campesinos!,"Hold On Now, Youngster",... And We Exhale And Roll Our Eyes In Unison,2:49,1.0,0,not-rated
1,The Steinways,Missed The Boat,... But You're Really Nice,0:17,2.0,undefined,never-rated
2,Titus Andronicus,The Most Lamentable Tragedy,[ intermission ],1:17,1.0,undefined,never-rated
3,A Giant Dog,Pile,& Rock & Roll,2:56,36.0,5,thumbs-up
4,Titus Andronicus,The Most Lamentable Tragedy,[ seven seconds ],0:07,NaN,undefined,never-rated


In [12]:
lyrics.head()

,lyrics
0,F
1,F
2,F
3,F
4,F


In [11]:
lyrics.columns = ['lyrics']

In [13]:
gpm['lyrics']=lyrics

In [14]:
gpm.head()

,artist,album,title,duration,playcount,rating,rating_interpretation,lyrics
0,Los Campesinos!,"Hold On Now, Youngster",... And We Exhale And Roll Our Eyes In Unison,2:49,1.0,0,not-rated,F
1,The Steinways,Missed The Boat,... But You're Really Nice,0:17,2.0,undefined,never-rated,F
2,Titus Andronicus,The Most Lamentable Tragedy,[ intermission ],1:17,1.0,undefined,never-rated,F
3,A Giant Dog,Pile,& Rock & Roll,2:56,36.0,5,thumbs-up,F
4,Titus Andronicus,The Most Lamentable Tragedy,[ seven seconds ],0:07,NaN,undefined,never-rated,F


Drop all the duplicates

In [25]:
gpm.drop_duplicates('title',inplace=True)

drop all that have mp3 in the title (for some reason...) and podcasts

In [29]:
gpm=gpm[~gpm['title'].str.contains('.mp3')]

In [42]:
gpm=gpm[~gpm['artist'].str.contains('Serial')]

In [44]:
gpm=gpm[~gpm['artist'].str.contains('truecrime')]

In [48]:
lyrics=gpm['lyrics']

In [53]:
gpm.drop('lyrics',axis=1,inplace=True)

In [57]:
gpm.reset_index(inplace=True)

In [59]:
gpm.drop('index',axis=1,inplace=True)

export cleaned gpm song list, including songs that missed

In [61]:
gpm.to_csv('GPM_song_list.csv')

In [64]:
lyrics=lyrics.reset_index()

In [66]:
lyrics.drop('index',axis=1,inplace=True)

export lyrics list including songs that missed

In [68]:
lyrics.to_csv('lyrics.csv',index_label='lyrics_id')

In [69]:
gpm.to_csv('GPM_song_list.csv',index_label='song_id')

In [70]:
gpm['lyrics']=lyrics

create song and lyrics list that excludes songs that were unsuccessful pulling lyrics

In [87]:
gpm_lyrics=gpm[gpm['lyrics'] !='F']

In [88]:
gpm_lyrics=gpm_lyrics.reset_index()

In [90]:
gpm_lyrics.drop('index',axis=1,inplace=True)

In [91]:
gpm_lyrics.head()

,artist,album,title,duration,playcount,rating,rating_interpretation,lyrics
0,Bon Iver,"22, A Million",00000 Million,3:54,1.0,undefined,never-rated,"Must've been forces, that took me on them wild..."
1,M83,"Dead Cities, Red Seas & Lost Ghosts",0078h,4:02,1.0,0,not-rated,I was someone else's girl Every time you look ...
2,The Magnetic Fields,50 Song Memoir,'02: Be True to Your Bar,3:34,NaN,undefined,never-rated,Sitting in bars and cafés Writing songs about ...
3,The Magnetic Fields,50 Song Memoir,'05: Never Again,3:19,NaN,undefined,never-rated,We'll walk in the rain Just having a day Never...
4,The Magnetic Fields,50 Song Memoir,"'06: ""Quotes""",2:17,NaN,undefined,never-rated,"For some of which you could have sued Oh, the ..."


In [92]:
gpm_lyrics.to_csv('song_lyrics.csv',index_label='song_id')

some of my lyrics lines had NaN values which is interesting. Songs that don't have lyrics, but did have songlyric webpages.

In [4]:
songlyrics=pd.read_csv('song_lyrics.csv',index_col='song_id')

In [11]:
songlyrics['lyrics'].fillna('F',inplace=True)

In [12]:
songlyrics['lyrics'].isnull().sum()

0

In [13]:
songlyrics=songlyrics[songlyrics['lyrics'] !='F']

In [16]:
songlyrics=songlyrics.reset_index()

In [19]:
songlyrics.drop('song_id',axis=1,inplace=True)

In [20]:
songlyrics.to_csv('song_lyrics.csv',index_label='song_id')